In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('sparkML').getOrCreate()

21/11/10 11:47:06 WARN Utils: Your hostname, laptop resolves to a loopback address: 127.0.1.1; using 10.19.10.34 instead (on interface wlo1)
21/11/10 11:47:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/10 11:47:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# read the dataset 
training = spark.read.csv('test1.csv', header=True, inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
# import VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [8]:
featureAssember = VectorAssembler(inputCols=['age','Experience'], outputCol="Independent Feature")

In [9]:
output = featureAssember.transform(training)

In [10]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [12]:
finalized_data = output.select("Independent Feature", "Salary")

In [13]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [14]:
# import Linear Regression
from pyspark.ml.regression import LinearRegression

In [15]:
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

In [16]:
#create a Linear Regression object
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='Salary')

In [17]:
# train the model
regressor = regressor.fit(train_data)

21/11/10 15:07:24 WARN Instrumentation: [1e467da8] regParam is zero, which might cause numerical instability and overfitting.
21/11/10 15:07:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/11/10 15:07:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/11/10 15:07:24 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/11/10 15:07:24 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [18]:
regressor.coefficients

DenseVector([-2087.7193, 3561.4035])

In [19]:
regressor.intercept

59140.35087715416

In [20]:
pred_results = regressor.evaluate(test_data)

In [21]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|18859.649122805153|
|         [29.0,4.0]| 20000|12842.105263165002|
+-------------------+------+------------------+



In [22]:
pred_results.meanSquaredError

33066174.20739037